# Fast sparse wishart

By `@jhmarcus`

Here I explore taking advantage the sparsity of the laplacian matrix (the precision matrix in the IBR model) to compute the log density of the Wishart distribution. `julia` seems to have nice linear algebra support for computing sparse cholesky decompositions which is exactly what we want.

# Background

Here is the `scipy.stats` implementation ...


```python
        log_det_x = np.zeros(x.shape[-1])
        scale_inv_x = np.zeros(x.shape)
        tr_scale_inv_x = np.zeros(x.shape[-1])
        for i in range(x.shape[-1]):
            _, log_det_x[i] = self._cholesky_logdet(x[:,:,i])
            scale_inv_x[:,:,i] = scipy.linalg.cho_solve((C, True), x[:,:,i])
            tr_scale_inv_x[i] = scale_inv_x[:,:,i].trace()

        # Log PDF
        out = ((0.5 * (df - dim - 1) * log_det_x - 0.5 * tr_scale_inv_x) -
               (0.5 * df * dim * _LOG_2 + 0.5 * df * log_det_scale +
                multigammaln(0.5*df, dim)))

```

Let $\mathbf{S}$ be the sample covariance matrix and $\mathbf{\Lambda}$ be the precision matrix i.e. $\mathbf{\Lambda} = \mathbf{\Sigma}^{-1}$. We can see the most expensive computations are computing ...

* $|\mathbf{S}|$ which is only computed once 
* $|\mathbf{\Lambda}^{-1}| = \frac{1}{|\mathbf{\Lambda}|}$ which computed each iteration of mcmc or optimization

Note that $\mathbf{S}$ is not sparse.

## Imports / Configuration

In [220]:
using Distributions
using LightGraphs

# set seed 
srand(1990) 

MersenneTwister(UInt32[0x000007c6], Base.dSFMT.DSFMT_state(Int32[-863370057, 1073434926, 174807504, 1073178214, 1034078006, 1073367208, -450309519, 1073714622, -24949693, 1073723854  …  1357943618, 1073343485, 520554461, 1072703876, 231516093, -1022460827, -640665436, -1160497238, 382, 0]), [1.95522, 1.46572, 1.97599, 1.41763, 1.00167, 1.23093, 1.10771, 1.07047, 1.60926, 1.71768  …  1.96975, 1.25279, 1.88343, 1.84652, 1.52945, 1.68392, 1.0424, 1.92873, 1.41223, 1.61294], 382)

## Exploration

Setup the precision matrix as the graph laplacian of a simple square grid ...

In [230]:
n_row = 200 # number of rows in grid
n_col = 200 # number of cols in grid
n = n_row * n_col # total number of nodes
G = Grid([n_row, n_col]) # create the grid
Q = laplacian_matrix(G) # compute graph laplacian
QQt = Q * Q' # Using the Hanks 2016 precision

40000×40000 SparseMatrixCSC{Int64,Int64} with 516004 stored entries:
  [1    ,     1]  =  6
  [2    ,     1]  =  -5
  [3    ,     1]  =  1
  [201  ,     1]  =  -5
  [202  ,     1]  =  2
  [401  ,     1]  =  1
  [1    ,     2]  =  -5
  [2    ,     2]  =  12
  [3    ,     2]  =  -6
  [4    ,     2]  =  1
  ⋮
  [39800, 39999]  =  2
  [39997, 39999]  =  1
  [39998, 39999]  =  -6
  [39999, 39999]  =  12
  [40000, 39999]  =  -5
  [39600, 40000]  =  1
  [39799, 40000]  =  2
  [39800, 40000]  =  -5
  [39998, 40000]  =  1
  [39999, 40000]  =  -5
  [40000, 40000]  =  6

In [233]:
@time begin
    F = cholfact(QQt) # do sparse cholesky factorization
    d = logdet(F) # compute logdet of QQt
end

  0.401222 seconds (63 allocations: 82.157 MiB, 0.98% gc time)


92575.73400504058

This seems pretty fast for a covariance of dimension ...

In [234]:
size(QQt)

(40000, 40000)

when I tried this in python using the naive `np.linalg.chol` on a laplacian stored as a dense matrix it failed and it doesn't look like there is support for cholesky factorization of sparse matrices using `scipy.sparse`. The `R` `Matrix` library looks interesting to explore more as wraps the same `CHOLMOD` library